Imports

In [ ]:
import sys
from pathlib import Path
import numpy as np
import matplotlib as mpl
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
from diffpy.utils.parsers.loaddata import loadData
from bg_mpl_stylesheet.bg_mpl_stylesheet import bg_mpl_style

Stating the file name of the iPython notebook.

In [ ]:
nb_name = "xy_overview.ipynb"

Dictionary with plot settings.

In [ ]:
D_PLOT = dict(dpi=600,
              figsize=(8, 4),
              fs_labels=20,
              fs_ticks=14,
              xmin=1,
              xmax=10,
              )

Printing the dictionary with plot settings.

In [ ]:
print(f"{80*'-'}\nThe following plot settings were found in the top of the "
      f"{nb_name} file:")
for k in list(D_PLOT.keys()):
      if len(k) < 8:
            print(f"\t{k}\t\t{D_PLOT[k]}")
      else:
            print(f"\t{k}\t{D_PLOT[k]}")
print(f"{80*'-'}\nPlease change the settings and rerun the code if neccessary.")

Checking whether code is executed from iPython notebook or terminal to be able 
to exit code properly.

In [ ]:
def is_nb():
    shell = get_ipython().__class__.__name__
    if shell == "ZMQInteractiveShell":
        nb_bool = True
    else:
        nb_bool = False

    return nb_bool

Checking whether 'data' folder for data files exists.

In [ ]:
data_path = Path.cwd() / "data"
if not data_path.exists():
    data_path.mkdir()
    s = f"{80*'-'}\nA folder called 'data' has been created.\n"
    s += f"Please put your data files there and rerun the code.\n{80*'-'}"
    print(s)
    if is_nb():
        exit(keep_kernel=True)
    else:
        exit()

Checking whether any data files are present in the 'data' folder.

In [ ]:
data_files = list(data_path.glob("*.*"))
if len(data_files) == 0:
    s = f"{80*'-'}\nNo files were found in the 'data' folder.\n"
    s += f"Please put your data files there and rerun the code.\n{80*'-'}"
    print(s)
    if is_nb():
        exit(keep_kernel=True)
    else:
        exit()

Checking that only one file extension is present among the data files in the 
'data' folder.

In [ ]:
suffixes = []
for f in data_files:
    if f.suffix not in suffixes:
        suffixes.append(f.suffix)
if len(suffixes) > 1:
    print(f"{80*'-'}\nThe following file extensions were found in the "
          f"'{data_path.name}' folder:")
    for suffix in suffixes:
        print(f"\t{suffix}")
    s = f"{80*'-'}\nPlease review the '{data_path.name}' folder to ensure "
    s += f"that only one file extension is\npresent.\n{80*'-'}"
    print(s)
    if is_nb():
        exit(keep_kernel=True)
    else:
        exit()
else:
    D_PLOT["suffix"] = suffixes[0].strip(".")

Updating dictionary with plot settings with labels according to the file 
extension of the data files.

In [ ]:
D_PLOT["xlabel"] = "scan number"
if suffixes[0] == ".gr":
    D_PLOT["ylabel"] = "$r\;[\mathrm{\AA}]$"
    D_PLOT["cbarlabel"] = "$G\;[\mathrm{\AA}^{-2}]$"
elif suffixes[0] == ".fq":
    D_PLOT["ylabel"] = "$Q\;[\mathrm{\AA}^{-1}]$"
    D_PLOT["cbarlabel"] = "$F\;[\mathrm{\AA}^{-1}]$"
else:
    D_PLOT["ylabel"] = "$Q\;[\mathrm{\AA}^{-1}]$"
    D_PLOT["cbarlabel"] = "$I\;[\mathrm{arb.\;u.}]$"

Collecting x array from first data file and stacking y arrays into one array.

In [ ]:
for i, f in enumerate(data_files):
    data = loadData(f)
    x, y = data[:, 0], data[:, 1]
    if i == 0:
        array = y
    else:
        array = np.column_stack((array, y))

Function to obtain indices for min and max values of array.

In [ ]:
def get_indices(array, min_value, max_value):
    min_index, max_index = None, None
    for i, e in enumerate(array):
        if e >= min_value:
            min_index = i
            break
    for i, e in enumerate(array):
        if e >= max_value:
            max_index = i
            break
    if isinstance(min_index, type(None)):
        min_index = 0
    if isinstance(max_index, type(None)):
        max_index = len(array) - 1

    return min_index, max_index

Shaping x array and stacked y array.

In [ ]:
xmin_index, xmax_index = get_indices(x, D_PLOT["xmin"], D_PLOT["xmax"])
if xmax_index < len(x) - 1:
    x, array = x[xmin_index:xmax_index + 1], array[xmin_index:xmax_index + 1, :]
else:
    x, array = x[xmin_index:], array[xmin_index:, :]
D_PLOT["vmin"], D_PLOT["vmax"] = np.amin(array), np.amax(array)

Function for shifting (diverging) colormap, such thast white corresponds to zero
even if data is min and max is not symmetric around zero, i.e., if min != - max.

In [ ]:
def shiftedColorMap(cmap, start=0, midpoint=0.5, stop=1.0, name='shiftedcmap'):
    '''
    Function to offset the "center" of a colormap. Useful for
    data with a negative min and positive max and you want the
    middle of the colormap's dynamic range to be at zero.

    Input
    -----
      cmap : The matplotlib colormap to be altered
      start : Offset from lowest point in the colormap's range.
          Defaults to 0.0 (no lower offset). Should be between
          0.0 and `midpoint`.
      midpoint : The new center of the colormap. Defaults to
          0.5 (no shift). Should be between 0.0 and 1.0. In
          general, this should be  1 - vmax / (vmax + abs(vmin))
          For example if your data range from -15.0 to +5.0 and
          you want the center of the colormap at 0.0, `midpoint`
          should be set to  1 - 5/(5 + 15)) or 0.75
      stop : Offset from highest point in the colormap's range.
          Defaults to 1.0 (no upper offset). Should be between
          `midpoint` and 1.0.
    '''
    cdict = {
        'red': [],
        'green': [],
        'blue': [],
        'alpha': []
    }

    # regular index to compute the colors
    reg_index = np.linspace(start, stop, 257)

    # shifted index to match the data
    shift_index = np.hstack([
        np.linspace(0.0, midpoint, 128, endpoint=False),
        np.linspace(midpoint, 1.0, 129, endpoint=True)
    ])

    for ri, si in zip(reg_index, shift_index):
        r, g, b, a = cmap(ri)

        cdict['red'].append((si, r, r))
        cdict['green'].append((si, g, g))
        cdict['blue'].append((si, b, b))
        cdict['alpha'].append((si, a, a))

    newcmap = mcolors.LinearSegmentedColormap(name, cdict)
    mpl.colormaps.register(cmap=newcmap, force=True)

    return newcmap

Function to truncate (diverging) colormap to get the positive part.

In [ ]:
def truncate_colormap(cmap, minval=0.0, maxval=1.0, n=100):
    new_cmap = mcolors.LinearSegmentedColormap.from_list(
               'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name,
                                                   a=minval,
                                                   b=maxval
                                                   ),
               cmap(np.linspace(minval, maxval, n)))

    return new_cmap

Creating diverging colormap with user-defined colors, shrinking colormap to 
adapt to data min and max values to have white corresponding to zero, and 
truncating the diverging colormap to get its positive part.

In [ ]:
div_gradient = mcolors.LinearSegmentedColormap.from_list('div_gradient', (
                 # Edit this gradient at 
                 # https://eltos.github.io/gradient/#0B3C5D-0B3C5D-FFFFFF-B82601-B82601
                 (0.000, (0.043, 0.235, 0.365)),
                 (0.250, (0.200, 0.400, 0.500)),
                 (0.500, (1.000, 1.000, 1.000)),
                 (0.750, (0.850, 0.200, 0.100)),
                 (1.000, (0.722, 0.149, 0.004))))
mp = 1 - D_PLOT["vmax"] / (D_PLOT["vmax"] + abs(D_PLOT["vmin"]))
cmap_shrunk = shiftedColorMap(div_gradient, 
                              start=0, 
                              midpoint=mp, 
                              stop=1, 
                              name='shrunk',
                              )
cmap_trunc = truncate_colormap(div_gradient, 0.5, 1)                              

Updating dictionary with plot settings to include colormap.

In [ ]:
if suffixes[0] == ".gr":
    D_PLOT["cmap"] = cmap_shrunk
elif suffixes[0] == ".fq":
    D_PLOT["cmap"] = cmap_shrunk
else:
    D_PLOT["cmap"] = cmap_trunc   

Plot function.

In [ ]:
def plot(x, array, d, output_paths):
    plt.style.use(bg_mpl_style)
    fig, ax = plt.subplots(figsize=d["figsize"])
    im = ax.imshow(array,
                   interpolation="none",
                   aspect="auto",
                   extent=(0, array.shape[1], d["xmax"], d["xmin"]),
                   vmin=np.amin(array),
                   vmax=np.amax(array),
                   cmap=d["cmap"],
                   )
    ax.tick_params(axis="x",
                   top=True,
                   bottom=True,
                   labeltop=True,
                   labelbottom=False,
                   )
    ax.tick_params(axis="y",
                   left=True,
                   right=True,
                   labelleft=True,
                   labelright=False,
                   )                   
    ax.set_xlabel(d["xlabel"], fontsize=d["fs_labels"])
    ax.xaxis.set_label_position("top")
    ax.set_ylabel(ylabel=d["ylabel"], fontsize=d["fs_labels"])
    ax.minorticks_on()
    cbar = plt.colorbar(im)
    cbar.set_label(label=d["cbarlabel"], fontsize=d["fs_labels"])
    if d[f"cbarlabel"] == "$I\;[\mathrm{arb.\;u.}]$":
        cbar.formatter.set_powerlimits((0, 0))
        cbar.ax.yaxis.set_offset_position('left')
        cbar.update_ticks()
    outputname = f"{d['suffix']}_overview_xmin={d['xmin']}_xmax={d['xmax']}"
    for p in output_paths:
        print(f"\t{p.name}")
        plt.savefig(p / f"{outputname}.{p.name}", 
                    bbox_inches="tight",
                    dpi=d["dpi"],
                    )
    plt.close()

    return None

Creating plot directories if not already existing.

In [ ]:
plot_folders = ["png", "pdf", "svg"]
plot_paths = [Path.cwd() / folder for folder in plot_folders]
for p in plot_paths:
    if not p.exists():
        p.mkdir()

Plotting.

In [ ]:
print(f"{80*'-'}\nPlotting...")
plot(x, array, D_PLOT, plot_paths)
print(f"Done.\n{80*'-'}\nPlease see the {plot_folders} folders.")